# PMML Generator - Example Code Below

Using JPMML package and data from hadoop, how can we develop a model that predicts billing outcomes on the Comcast eIVR?

In [ ]:
# from sklearn_pandas import DataFrameMapper
# default_mapper = DataFrameMapper([(i, None) for i in iris.feature_names + ['Species']])

# from sklearn2pmml import sklearn2pmml
# sklearn2pmml(estimator=clf, 
#              mapper=default_mapper, 
#              pmml="D:/workspace/IrisClassificationTree.pmml")

In [127]:
from sklearn_pandas import DataFrameMapper
from sklearn2pmml import sklearn2pmml
from sklearn import pipeline
import pandas as pd

df = pd.read_csv('c:/users/jswortz/comcastBilling3.csv', low_memory=False, na_values = ["N/A", "NA"])

stringFeatures = ['outageflag', 'install', 'broadcastfunction', 'cancelledappt','boradcaststatus',
                  'anilearning', 'accountstatus']
normalizeFeatures = ['nonpaydiscoii', 'delinq_days']
from sklearn.preprocessing import LabelBinarizer, LabelEncoder, StandardScaler
# df = df.head(10000)
# df.outageflag = df.outageflag.replace(True, "TRUE").replace(False, "FALSE").astype(str)
# df.install = df.install.replace(True, "TRUE").replace(False, "FALSE").astype(str)
# df.broadcastfunction = df.broadcastfunction.replace(True, "TRUE").replace(False, "FALSE").astype(str)
# df.boradcaststatus = df.boradcaststatus.replace(True, "TRUE").replace(False, "FALSE").astype(str)
# df.cancelledappt = df.cancelledappt.replace(True, "TRUE").replace(False, "FALSE").astype(str)
# df.anilearning = df.anilearning.replace(True, "TRUE").replace(False, "FALSE").astype(str)
# df.accountstatus = df.accountstatus.replace(True, "TRUE").replace(False, "FALSE").astype(str)
df.dtypes

currentbalance                       float64
daysdelinquent                         int64
previousdueamount                    float64
previouspaymentamount                float64
statmentbalance                      float64
billingpin                            object
cardpaymentrestricted                   bool
bankpaymentrestricted                   bool
existingrecurringpaymentonaccount       bool
balancedue                           float64
pastduebalance                       float64
pendingpaymentamount                 float64
projectedbalancedue                  float64
recentpaymentamount                  float64
statementbalance                     float64
outageflag                            object
install                               object
broadcastfunction                     object
boradcaststatus                       object
cancelledappt                         object
anilearning                           object
accountstatus                         object
nonpaydisc

In [134]:
# df.cardPaymentRestricted = df.cardPaymentRestricted.astype(str)
df.cardpaymentrestricted = df.cardpaymentrestricted.astype(str)
df.existingrecurringpaymentonaccount = df.existingrecurringpaymentonaccount.astype(str)
df.due_diff = df.due_diff.astype(str)

# df.billingPin = df.billingPin.astype(str)

df = df.replace(np.nan,'NULLZ', regex=True)

df = df.replace("", "BLANK")

df.dtypes
df

# pd.pivot_table(df,values=1,index=['outagflag'],columns=['billing'],aggfunc=np.average)

,currentbalance,daysdelinquent,previousdueamount,previouspaymentamount,statmentbalance,billingpin,cardpaymentrestricted,bankpaymentrestricted,existingrecurringpaymentonaccount,balancedue,...,accountstatus,nonpaydiscoii,delinq_days,billing,due_diff,prev_pay_diff,bal_due_diff,past_due_diff,rec_pay_diff,promise_diff
0,419.85,30,208.62,208.60,419.85,BLANK,False,False,False,419.85,...,ACTIVE,30,30,0,17,-42,17,-9999,-9999,-9999
1,95.17,0,0.00,139.34,95.17,201,False,False,False,95.17,...,ACTIVE,0,0,1,-1,-51,-1,-9999,-9999,-9999
2,148.04,0,0.00,182.67,156.98,240,False,False,False,156.98,...,ACTIVE,0,0,0,9,-30,9,-9999,-9999,-9999
3,0.00,0,0.00,183.82,183.82,BLANK,False,False,False,0.00,...,ACTIVE,0,0,1,-4,-5,-4,-9999,-9999,-9999
4,66.70,0,0.00,73.51,66.70,220,False,False,False,66.70,...,INACTIVE,0,0,0,11,-31,11,-9999,-9999,-9999
5,480.74,30,174.97,100.00,542.76,BLANK,False,False,False,442.76,...,ACTIVE,30,30,0,-4,-18,-4,-9999,-9999,-9999
6,166.14,0,0.00,131.51,166.14,101,False,False,False,166.14,...,ACTIVE,0,0,1,20,-11,21,-9999,-9999,-9999
7,170.97,0,0.00,143.48,164.11,141,False,False,False,162.47,...,BLANK,0,0,0,-1,-35,0,-9999,-9999,-9999
8,512.63,42,251.78,275.23,781.87,400,False,False,False,506.64,...,ACTIVE,42,30,0,8,-12,8,-9999,-9999,-9999
9,198.80,0,0.00,96.28,198.80,440,False,False,False,198.80,...,ACTIVE,0,0,0,9,-20,9,-9999,-9999,-9999


In [159]:
###starting over - using examples from main.py
from sklearn.preprocessing import LabelBinarizer, LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.svm import SVC
from sklearn_pandas import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn2pmml import sklearn2pmml
from sklearn.metrics import confusion_matrix
from sklearn.ensemble.forest import RandomForestClassifier
from sklearn.tree.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble.gradient_boosting import GradientBoostingClassifier, GradientBoostingRegressor
from sklearn2pmml.decoration import ContinuousDomain
from sklearn.externals import joblib
from sklearn.model_selection import train_test_split
from sklearn_pandas import cross_val_score
from sklearn.model_selection import KFold, cross_val_score
from sklearn.neural_network import MLPClassifier, MLPRegressor
from sklearn.linear_model.coordinate_descent import ElasticNetCV, LassoCV


k_fold = KFold(n_splits=3)

train, test = train_test_split(df, test_size = 0.1)


def store_pkl(obj, name):
	joblib.dump(obj, name, compress = 9)

mapper3 = DataFrameMapper([
        ("currentbalance", ContinuousDomain()),
        ("daysdelinquent", ContinuousDomain()),
        ("previouspaymentamount", ContinuousDomain()),
        ("statmentbalance", ContinuousDomain()),
        ("existingrecurringpaymentonaccount", LabelEncoder()),
        ("balancedue", ContinuousDomain()),
        ("pendingpaymentamount", ContinuousDomain()),
        ("pastduebalance", ContinuousDomain()),
        ("projectedbalancedue", ContinuousDomain()),
        ("recentpaymentamount", ContinuousDomain()),
        ("outageflag", LabelBinarizer()),
        ("install", LabelBinarizer()),
        ("broadcastfunction", LabelEncoder()),
        ("boradcaststatus", LabelBinarizer()),
        ("cancelledappt", LabelBinarizer()),
        ("anilearning", LabelBinarizer()),
        ("accountstatus", LabelBinarizer()),
        (["nonpaydiscoii"], StandardScaler()),
        (["delinq_days"], StandardScaler()),
#         ("due_diff", ContinuousDomain()),
#         ("bal_due_diff", ContinuousDomain()),
#         ("promise_diff", ContinuousDomain()),
        ("billing", None)
    ])

fitMap = mapper3.fit_transform(train)



# store_pkl(mapper3,"ccBillingMap.pkl")

print(fitMap.shape)

fmX = fitMap[:, 0:36]
fmY = fitMap[:, 36]

fmY = fmY.astype(int)

clf = SVC(C=0.1, cache_size=200, class_weight="balanced", coef0=0.1,
    decision_function_shape='ovr', degree=5, gamma='auto', kernel='rbf',
    max_iter=-1, probability=False, random_state=12, shrinking=True,
    tol=1, verbose=False)

clf2 = RandomForestClassifier(n_estimators=20, criterion='gini', max_depth=None, min_samples_split=10, 
                              min_samples_leaf=1, min_weight_fraction_leaf=0.0, max_features='auto', 
                              max_leaf_nodes=None, min_impurity_split=0.07, bootstrap=True, oob_score=False, 
                              n_jobs=-1, random_state=12, verbose=0, warm_start=False, class_weight="balanced")

clf3 = GradientBoostingClassifier(random_state = 13, loss = "exponential", init = None)

clf5 = ElasticNetCV(random_state = 13, l1_ratio = 1)
# clf4 = MLPClassifier(activation = "tanh", hidden_layer_sizes = (26,12,8,4,2,), random_state = 13, 
#                      tol = 0.0000001, max_iter = 10000, learning_rate='adaptive', solver='sgd')

clf3.fit(fmX,fmY)

# pipe = pipeline.Pipeline([
#         ('mapper', mapper3),
#         ('clf', SVC(class_weight = "balanced"))
#     ])

# cv1 = cross_val_score(clf, fmX, fmY, cv=k_fold, scoring='roc_auc')

cv2 = cross_val_score(clf3, fmX, fmY, cv=k_fold, scoring='roc_auc')

# print(cv1)

print(cv2)

fittedTest = mapper3.transform(test)

xTest = fittedTest[:, 0:36]
yTest = fittedTest[:, 36]

predicted = clf3.predict(xTest)


# store_pkl(clf,"DecisionTreeAudit.pkl")

# sklearn2pmml(estimator=clf2, 
#              mapper=mapper3, 
#              pmml="c:\\users\\jswortz\\example6.pmml")


(45000L, 37L)
[ 0.7   0.69  0.71]


In [56]:

predData = zip(clf2.predict_proba(xTest),yTest)

predData

[(array([ 0.79,  0.21]), 0.0),
 (array([ 0.34,  0.66]), 0.0),
 (array([ 0.59,  0.41]), 0.0),
 (array([ 0.94,  0.06]), 0.0),
 (array([ 0.49,  0.51]), 0.0),
 (array([ 0.68,  0.32]), 1.0),
 (array([ 0.45,  0.55]), 1.0),
 (array([ 0.51,  0.49]), 0.0),
 (array([ 0.6,  0.4]), 0.0),
 (array([ 0.4,  0.6]), 0.0),
 (array([ 0.43,  0.57]), 1.0),
 (array([ 0.58,  0.42]), 0.0),
 (array([ 0.85,  0.15]), 0.0),
 (array([ 0.67,  0.33]), 1.0),
 (array([ 0.65,  0.35]), 1.0),
 (array([ 0.69,  0.31]), 1.0),
 (array([ 0.74,  0.26]), 1.0),
 (array([ 0.76,  0.24]), 0.0),
 (array([ 0.73,  0.27]), 1.0),
 (array([ 0.82,  0.18]), 0.0),
 (array([ 0.53,  0.47]), 1.0),
 (array([ 0.38,  0.62]), 0.0),
 (array([ 0.6,  0.4]), 1.0),
 (array([ 0.47,  0.53]), 0.0),
 (array([ 0.74,  0.26]), 0.0),
 (array([ 0.81,  0.19]), 0.0),
 (array([ 0.5,  0.5]), 1.0),
 (array([ 0.75,  0.25]), 0.0),
 (array([ 0.89,  0.11]), 0.0),
 (array([ 0.37,  0.63]), 1.0),
 (array([ 0.3,  0.7]), 1.0),
 (array([ 0.73,  0.27]), 0.0),
 (array([ 0.87,  0

,outageflag,install,broadcastfunction,boradcaststatus,cancelledappt,anilearning,accountstatus,nonpaydiscoii,delinq_days,p_p_billing,p_p_p_p_p_billing
2284,FALSE,MISSING,MISSING,MISSING,MISSING,NOT_FOUND,ACTIVE,65,3.000000e+01,0,0
6020,FALSE,FALSE,Array,FOUND,FALSE,NOT_FOUND,ACTIVE,0,1.000000e+00,0,0
9271,FALSE,MISSING,MISSING,MISSING,MISSING,NOT_FOUND,ACTIVE,52,3.000000e+01,0,0
8569,FALSE,MISSING,MISSING,MISSING,MISSING,NOT_FOUND,INACTIVE,48,3.000000e+01,0,0
5726,FALSE,FALSE,Array,NOT_FOUND,FALSE,NOT_FOUND,ACTIVE,42,3.000000e+01,0,0
6142,FALSE,FALSE,Array,NOT_FOUND,FALSE,NOT_FOUND,ACTIVE,0,1.000000e+00,0,0
4066,FALSE,FALSE,MISSING,MISSING,FALSE,NOT_FOUND,ACTIVE,0,7.580000e+09,0,0
2516,FALSE,MISSING,MISSING,MISSING,MISSING,NOT_FOUND,ACTIVE,67,3.000000e+01,0,0
4297,FALSE,FALSE,Array,NOT_FOUND,FALSE,NOT_FOUND,ACTIVE,0,3.000000e+01,0,0
4866,FALSE,FALSE,Array,NOT_FOUND,FALSE,NOT_FOUND,ACTIVE,0,0.000000e+00,0,0


In [60]:
from sklearn.preprocessing import LabelBinarizer, LabelEncoder, StandardScaler, OneHotEncoder
from sklearn.svm import SVC
from sklearn_pandas import cross_val_score
from sklearn.model_selection import train_test_split
from sklearn2pmml import sklearn2pmml
from sklearn.metrics import confusion_matrix
from sklearn.ensemble.forest import RandomForestClassifier


# df = df.head(100)

train, test = train_test_split(df, test_size = 0.2)

mapper = DataFrameMapper(
    [(x, LabelBinarizer()) for x in stringFeatures] +
    [(x, StandardScaler()) for x in normalizeFeatures]  +
     [(['billing'], None)])

mapper2 = DataFrameMapper(
     [('nonpaydiscoii', StandardScaler()),
     ('billing', None)])

pipe = pipeline.Pipeline([
        ('mapper', mapper),
        ('clf', SVC())
    ])

fittedMapper = mapper2.fit_transform(train)



clf = SVC()
#forest = RandomForestClassifier()


x = fittedMapper[:, 0]
y = fittedMapper[:, 1]
fittedMapper[1]

clf.fit(x,y)

# fittedTest = mapper.transform(test)

# xTest = fittedTest[:, 0:14]
# yTest = fittedTest[:, 15]

# predicted = forest.predict(xTest)

# sklearn2pmml(estimator=clf, 
#              mapper=mapper, 
#              pmml="c:\\users\\jswortz\\example.pmml")



C:\Anaconda2\lib\site-packages\sklearn\preprocessing\data.py:586: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
C:\Anaconda2\lib\site-packages\sklearn\preprocessing\data.py:649: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  warnings.warn(DEPRECATION_MSG_1D, DeprecationWarning)
C:\Anaconda2\lib\site-packages\sklearn\utils\validation.py:395: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and will raise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contain

ValueError: Found input variables with inconsistent numbers of samples: [1, 8000]

In [54]:
sklearn2pmml(estimator=clf, 
             mapper=mapper2, 
             pmml="c:\\users\\jswortz\\example.pmml", debug = True)

# from sklearn.externals import joblib

# joblib.dump(mapper, "mapper.pkl", compress = 9)
# joblib.dump(clf, "estimator.pkl", compress = 9)

('python: ', '2.7.12')
('sklearn: ', '0.18.1')
('sklearn.externals.joblib:', '0.10.3')
('sklearn_pandas: ', '1.2.0')
('sklearn2pmml: ', '0.12.0')
java -cp C:\Users\jswortz\AppData\Roaming\Python\Python27\site-packages\sklearn2pmml\resources\guava-19.0.jar;C:\Users\jswortz\AppData\Roaming\Python\Python27\site-packages\sklearn2pmml\resources\istack-commons-runtime-2.21.jar;C:\Users\jswortz\AppData\Roaming\Python\Python27\site-packages\sklearn2pmml\resources\jaxb-core-2.2.11.jar;C:\Users\jswortz\AppData\Roaming\Python\Python27\site-packages\sklearn2pmml\resources\jaxb-runtime-2.2.11.jar;C:\Users\jswortz\AppData\Roaming\Python\Python27\site-packages\sklearn2pmml\resources\jcommander-1.48.jar;C:\Users\jswortz\AppData\Roaming\Python\Python27\site-packages\sklearn2pmml\resources\jpmml-converter-1.1.1.jar;C:\Users\jswortz\AppData\Roaming\Python\Python27\site-packages\sklearn2pmml\resources\jpmml-sklearn-1.1.3.jar;C:\Users\jswortz\AppData\Roaming\Python\Python27\site-packages\sklearn2pmml\resou

CalledProcessError: Command '['java', '-cp', 'C:\\Users\\jswortz\\AppData\\Roaming\\Python\\Python27\\site-packages\\sklearn2pmml\\resources\\guava-19.0.jar;C:\\Users\\jswortz\\AppData\\Roaming\\Python\\Python27\\site-packages\\sklearn2pmml\\resources\\istack-commons-runtime-2.21.jar;C:\\Users\\jswortz\\AppData\\Roaming\\Python\\Python27\\site-packages\\sklearn2pmml\\resources\\jaxb-core-2.2.11.jar;C:\\Users\\jswortz\\AppData\\Roaming\\Python\\Python27\\site-packages\\sklearn2pmml\\resources\\jaxb-runtime-2.2.11.jar;C:\\Users\\jswortz\\AppData\\Roaming\\Python\\Python27\\site-packages\\sklearn2pmml\\resources\\jcommander-1.48.jar;C:\\Users\\jswortz\\AppData\\Roaming\\Python\\Python27\\site-packages\\sklearn2pmml\\resources\\jpmml-converter-1.1.1.jar;C:\\Users\\jswortz\\AppData\\Roaming\\Python\\Python27\\site-packages\\sklearn2pmml\\resources\\jpmml-sklearn-1.1.3.jar;C:\\Users\\jswortz\\AppData\\Roaming\\Python\\Python27\\site-packages\\sklearn2pmml\\resources\\jpmml-xgboost-1.1.1.jar;

In [125]:
import itertools
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_fscore_support
from sklearn import svm, datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

class_names = ["noBilling","billing"]

def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, cm[i, j],
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

# Compute confusion matrix
cnf_matrix = confusion_matrix(yTest, predicted)
np.set_printoptions(precision=2)

# Plot non-normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=class_names,
                      title='Confusion matrix, without normalization')

# Plot normalized confusion matrix
plt.figure()
plot_confusion_matrix(cnf_matrix, classes=class_names, normalize=True,
                      title='Normalized confusion matrix')

plt.show()

metrics = precision_recall_fscore_support(yTest, predicted, average='binary')

print(metrics)

Confusion matrix, without normalization
[[1958 1157]
 [ 632 1253]]
Normalized confusion matrix
[[ 0.63  0.37]
 [ 0.34  0.66]]
(0.51991701244813282, 0.66472148541114062, 0.58346915017462164, None)


In [197]:
import pandas

iris_df = pandas.read_csv("iris.csv")
iris_df
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn_pandas import DataFrameMapper
from sklearn2pmml.decoration import ContinuousDomain

iris_mapper = DataFrameMapper([
    (["SepalLength", "SepalWidth", "PetalLength", "PetalWidth"], [ContinuousDomain(), StandardScaler(), PCA(n_components = 3)]),
    ("Name", None)
])

iris = iris_mapper.fit_transform(iris_df)

from sklearn.ensemble.forest import RandomForestClassifier

iris_X = iris[:, 0:3]
iris_y = iris[:, 3]

iris_forest = RandomForestClassifier(min_samples_leaf = 5)
iris_forest.fit(iris_X, iris_y)

sklearn2pmml(estimator=iris_forest, 
             mapper=iris_mapper, 
             pmml="c:\\users\\jswortz\\example.pmml", debug = True)


('python: ', '2.7.12')
('sklearn: ', '0.18.1')
('sklearn.externals.joblib:', '0.10.3')
('sklearn_pandas: ', '1.2.0')
('sklearn2pmml: ', '0.12.0')
java -cp C:\Users\jswortz\AppData\Roaming\Python\Python27\site-packages\sklearn2pmml\resources\guava-19.0.jar;C:\Users\jswortz\AppData\Roaming\Python\Python27\site-packages\sklearn2pmml\resources\istack-commons-runtime-2.21.jar;C:\Users\jswortz\AppData\Roaming\Python\Python27\site-packages\sklearn2pmml\resources\jaxb-core-2.2.11.jar;C:\Users\jswortz\AppData\Roaming\Python\Python27\site-packages\sklearn2pmml\resources\jaxb-runtime-2.2.11.jar;C:\Users\jswortz\AppData\Roaming\Python\Python27\site-packages\sklearn2pmml\resources\jcommander-1.48.jar;C:\Users\jswortz\AppData\Roaming\Python\Python27\site-packages\sklearn2pmml\resources\jpmml-converter-1.1.1.jar;C:\Users\jswortz\AppData\Roaming\Python\Python27\site-packages\sklearn2pmml\resources\jpmml-sklearn-1.1.3.jar;C:\Users\jswortz\AppData\Roaming\Python\Python27\site-packages\sklearn2pmml\resou

In [193]:
#
# Binary classification
#

audit_df = pd.read_csv("Audit.csv", na_values = ["N/A", "NA"])

print(audit_df.dtypes)

audit_df["Deductions"] = audit_df["Deductions"].replace(True, "TRUE").replace(False, "FALSE").astype(str)

print(audit_df.dtypes)

audit_mapper = DataFrameMapper([
    ("Age", ContinuousDomain()),
    ("Employment", LabelBinarizer()),
    ("Education", LabelBinarizer()),
    ("Marital", LabelBinarizer()),
    ("Occupation", LabelBinarizer()),
    ("Income", ContinuousDomain()),
    ("Gender", LabelEncoder()),
    ("Deductions", LabelEncoder()),
    ("Hours", ContinuousDomain()),
    ("Adjusted", None)
])

audit = audit_mapper.fit_transform(audit_df)

clf = SVC()

print(audit.shape)

x = audit[:,0:48]
y = audit[:, 48]

clf.fit(x,y)


sklearn2pmml(estimator=clf, 
             mapper=audit_mapper, 
             pmml="c:\\users\\jswortz\\example2.pmml")

ID              int64
Age             int64
Employment     object
Education      object
Marital        object
Occupation     object
Income        float64
Gender         object
Deductions    float64
Hours           int64
Account        object
Adjustment      int64
Adjusted        int64
dtype: object
ID              int64
Age             int64
Employment     object
Education      object
Marital        object
Occupation     object
Income        float64
Gender         object
Deductions     object
Hours           int64
Account        object
Adjustment      int64
Adjusted        int64
dtype: object
(1859L, 49L)


In [195]:
from sklearn.decomposition import PCA
from sklearn.preprocessing import StandardScaler
from sklearn_pandas import DataFrameMapper
from sklearn2pmml.decoration import ContinuousDomain

